# Using this Notebook to create Lambda Layers and deploy Lambda app to run the SAmazon SageMaker Endpoint
---

## Run Docker to build OpenCV Lambda Layers:

In [ ]:
!pushd ../docker
!docker build --tag aws-lambda-layers:latest .
!docker run -rm -it -v $(pwd):/layers aws-lambda-layers cp /packages/cv2-python37.zip /layers
!popd

## Push the OpenCV Lambda Layers zip to AWS Lambda and create the Lambda Layer:

- Need to provide the Amazon S3 bucket location to upload the Lambda script:
    `s3://<BUCKET>/<PATH/TO/STORE/ARTIFACTS>`

In [ ]:
!aws s3 cp layers/cv2-python37.zip s3://<BUCKET>/<PATH/TO/STORE/ARTIFACTS>
!aws lambda publish-layer-version --layer-name cv2 --description "Open CV" --content S3Bucket=<BUCKET>,S3Key=<PATH/TO/STORE/ARTIFACTS>/cv2-python37.zip --compatible-runtimes python3.7

## Deploy the Lambda function and connect the OpenCV Lambda Layers:

- Need to provide the Amazon S3 bucket location to upload the Lambda script:
    `s3://<BUCKET>/<PATH/TO/STORE/FUNCTION>`

In [ ]:
!pushd ../lambda
!zip app.zip app.py
!aws s3 cp app.zip s3://<BUCKET>/<PATH/TO/STORE/FUNCTION>
!aws lambda create-function --function-name yolov5-lambda --handler app.lambda_handler --region us-east-1 --runtime python3.7 --environment ""Variables={BUCKET_NAME=$BUCKET_NAME,S3_KEY=$S3_KEY}"" --code S3Bucket=<BUCKET>,S3Key="<PATH/TO/STORE/FUNCTION/app.zip>"
!popd
!aws lambda update-function-configuration --function-name yolov5-lambda --layers cv2